In [1]:
import os
import time
import tensorflow as tf
import scipy.io as sio 
from keras.applications.resnet50 import ResNet50
from keras.applications.mobilenet import MobileNet
from keras import models, Sequential
import keras


C:\Users\USER\anaconda3\envs\MNIST\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\USER\anaconda3\envs\MNIST\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\USER\anaconda3\envs\MNIST\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\USER\anaconda3\envs\MNIST\lib\site-packages\tensorflow\python\frame

In [2]:
tf.test.is_gpu_available()

False

In [3]:

def one_hot_matrix(labels, C):
    
    ### START CODE HERE ###
    
    # Create a tf.constant equal to C (depth), name it 'C'.
    depth = tf.constant(C, name = "C")
    
    # Use tf.one_hot, be careful with the axis (approx. 1 line)
    one_hot_matrix = tf.one_hot(labels, depth, axis=0)
    
    # Create the session
    sess = tf.Session()
    
    # Run the session
    one_hot = sess.run(one_hot_matrix)
    
    # Close the session.
    sess.close() 
    
    return one_hot

In [4]:
import random
file_path = os.getcwd() + '\\train'
label_name = os.listdir(file_path)
class_num = 3

matfn='data_preprocessed.mat' 
data=sio.loadmat(matfn) 


X_train = data['X_data'] 
X_train = X_train / 255
Y_train = data['Y_data']

"""
X_val = X_train[4200:]
Y_val = Y_train[4200:]

X_train = X_train[:4200]
Y_train = Y_train[:4200]
"""

print(X_train.shape)
print(Y_train)

(6000, 224, 224, 3)
[[0 0 0 ... 2 2 2]]


In [5]:
from tensorflow import train
data_num = X_train.shape[0] 

Y_train = one_hot_matrix(Y_train, class_num)
Y_train = Y_train.T
Y_train = Y_train.reshape(data_num,class_num)

from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau

# early_stopping : val_loss 連續不降低時，結束訓練
early_stopping = EarlyStopping(monitor='val_loss', patience=15, verbose=1,restore_best_weights=True,mode='auto')
# reduce_lr : val_loss 連續不降低時，減少learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, mode='auto',
                            epsilon=0.0001, cooldown=0, min_lr=0)

checkpoint = ModelCheckpoint('MobileNet_transfer_preprocess.h5', monitor='val_loss', verbose=1,
    save_best_only=True, mode='auto', period=1)

model = Sequential()
based_model = models.load_model('MobileNet_1.h5')
model.add(based_model)
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

based_model.trainable = True


model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=["accuracy"])
history = model.fit(x=X_train, y=Y_train, validation_split=0.05, epochs=100, batch_size=64, callbacks=[early_stopping,checkpoint,reduce_lr])

complete_time=time.strftime("%Y_%m_%d %H_%M_%S", time.localtime()) 
# model.save('MobileNet_'+str(complete_time)+'.h5')


Instructions for updating:
Colocations handled automatically by placer.


C:\Users\USER\anaconda3\envs\MNIST\lib\site-packages\keras\callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 5700 samples, validate on 300 samples
Epoch 1/100
5700/5700 [==============================] - 2079s 365ms/step - loss: 0.7723 - acc: 0.6033 - val_loss: 1.2250 - val_acc: 0.0000e+00

Epoch 00001: val_loss improved from inf to 1.22498, saving model to MobileNet_transfer_preprocess.h5
Epoch 2/100
1856/5700 [========>.....................] - ETA: 22:28 - loss: 0.6147 - acc: 0.6460

KeyboardInterrupt: 

In [ ]:
model.predict(X_train)